In [2]:
import pandas as pd

# === 샘플 데이터 ===
orders = pd.DataFrame({
    'order_id': [1, 2, 3, 4, 5],
    'user_id': [101, 102, 101, 103, 102],
    'city': ['Seoul', 'Busan', 'Seoul', 'Seoul', 'Busan'],
    'amount': [10000, 15000, 20000, 5000, 25000]
})

users = pd.DataFrame({
    'user_id': [101, 102, 103],
    'name': ['철수', '영희', '민수'],
    'grade': ['VIP', 'Normal', 'VIP']
})

print("=== orders ===")
print(orders)
print("\n=== users ===")
print(users)

=== orders ===
   order_id  user_id   city  amount
0         1      101  Seoul   10000
1         2      102  Busan   15000
2         3      101  Seoul   20000
3         4      103  Seoul    5000
4         5      102  Busan   25000

=== users ===
   user_id name   grade
0      101   철수     VIP
1      102   영희  Normal
2      103   민수     VIP


### 도시(city)별 총 매출 구하기

In [ ]:
# 도시별 총매출은 간단하게 그룹바이로 묶어서 sum을 진행하면 될꺼같다.
# 🐚 여기서 주의점은 df꼴로 나오기 위해서는 이중리스트를 사용하거나 as_index=False, reset_index()를 활용한다는것이다. 
# 🐚 이떄 ⭐️주의점은 이중리스트는 인덱스로 되긴함. 

# 🐚 orders['total_salse'] = orders.groupby('city',as_index=False)['amount'].sum() 오류 이유 : 그룹바이에서는 n행 1열로 줄이는데 지금 orders에서 새로 만드는 피처의 행렬크기와는 다르다.
# 🐚 따라서 transform으로 다 들가게 형 맞추거나 애초에 orders 테이블에 컬럼을 추가하는것이 아닌 새로운 테이블을 만들어도 된다. 
# 🐚 굳이 transform을 사용할떄는 그룹에 대한 집계함수 결과를 단일값과 비교하기 위해서 살리는것이다. 

# order에 피쳐 추가해서 넣는것. 
orders['total_salse'] = orders.groupby('city',as_index=False)['amount'].transform('sum') 
print(f"기존에서 만들기 :\n {orders}")

# 🐚 새로운 테이블 만들어서 사용하기 (지역만 묶기)
orders_total_sales = orders.groupby('city',as_index=False)['amount'].sum()
print(f"\n 새로운 테이블로 만들기 : \n{orders_total_sales}")

# 🐚 city과 user_id를 묶기 묶음이 하나의 셋이라서 또 다른 결과를 낳는다.
orders_total_sales = orders.groupby(['city','user_id'],as_index=False)['amount'].sum()
print(f"\n 새로운 테이블로 만들기 : \n{orders_total_sales}")

기존에서 만들기 :
    order_id  user_id   city  amount  해당_유저의_평균_주문액  total_salse
0         1      101  Seoul   10000          30000        35000
1         2      102  Busan   15000          40000        40000
2         3      101  Seoul   20000          30000        35000
3         4      103  Seoul    5000           5000        35000
4         5      102  Busan   25000          40000        40000

 새로운 테이블로 만들기 : 
    city  amount
0  Busan   40000
1  Seoul   35000


### 1번 결과를 DataFrame으로 만들고 컬럼명을 city, total로 변경

In [ ]:
# 🐚 컬럼명은 replace로 안되나? 그냥 새로 만들고 드랍해야해? -> No .rename(columns=({})) 사용  
# 🐚 replace는 내용물을 바꾸는 함수 rename은 컬럼,행렬 명을 바꾸는함수. 따라서 rows= , columns =를 정확히 명시해야한다. 
# 🐚  =을 안해서 원본에 반영을 안하는 참사가 존재했다. 
orders_total_sales = orders_total_sales.rename(columns=({'amount':'total'}))

,city,total
0,Busan,40000
1,Seoul,35000


### orders에 유저 이름(name) 붙이기

In [ ]:
import pandas as pd 
# 🐚 조건을 통해 특정만 붙이는방법 
orders_name = pd.merge(orders,users[['user_id','name']],how='left',on='user_id') 
print(orders_name)

# 🐚 무지성 조건없이 째로 붙이는 방법 .concat() axis를 통해 붙이는 방향 설정 중요
''' orders_total_sales_name = pd.concat([orders_total_sales,users[['name']]],axis=1)
orders_total_sales_name '''

   order_id  user_id   city  amount  total_salse name
0         1      101  Seoul   10000        35000   철수
1         2      102  Busan   15000        40000   영희
2         3      101  Seoul   20000        35000   철수
3         4      103  Seoul    5000        35000   민수
4         5      102  Busan   25000        40000   영희


" orders_total_sales_name = pd.concat([orders_total_sales,users[['name']]],axis=1)\norders_total_sales_name "

### 유저(user_id)별 총 주문액 구한 뒤, 유저 정보(users)와 합치기

In [ ]:
# 그룹바이를 통해 유저별로 묶은다음에 sum을 진행한다. 새로운 테이블을 만들어서 진행.
# 이후 usrs와 merge를 진행하는데 이거는 user_id를 기준으로 전체 합친다.

# 🐚 그냥 단순 user_id로만 묶으니까 city가 두번씩나오는데 이거는 왜그런거? city도 그룹으로 묶어 줘야하는건가? 자동으로 안묶이고 ? -> 어떤걸 더할지 안정함 ㅋㅋ groupby('')만했음 ['']이게 없었다.
user_group = orders.groupby('user_id',as_index=False)['amount'].sum()
user_group

users_usergroup = pd.merge(users,user_group,how='inner',on='user_id')
users_usergroup

,user_id,amount
0,101,30000
1,102,40000
2,103,5000


### orders에 "해당 유저의 평균 주문액" 컬럼 추가

In [3]:
# orders에 그룹바이를 id로 하고 각각의 평균 금액을 적어한다. 이때 컬럼추가이기에 피쳐수가 그룹바이 전과 동일해야하며 이는 transform을 이용하라는것이다.

orders['해당_유저의_평균_주문액'] = orders.groupby('user_id',as_index=False)['amount'].transform('mean')
orders

,order_id,user_id,city,amount,해당_유저의_평균_주문액
0,1,101,Seoul,10000,15000.0
1,2,102,Busan,15000,20000.0
2,3,101,Seoul,20000,15000.0
3,4,103,Seoul,5000,5000.0
4,5,102,Busan,25000,20000.0


### 5번 결과로 "평균 대비 비율" 컬럼 추가

In [ ]:
# 평균 컬럼 하나 만들어서 해당_유저의_평균_주문액랑 나누기 시켜야하나

orders['총주문액'] = orders.groupby('user_id',as_index=False)['amount'].transform('sum')
orders['평균대비_비율'] =  orders['해당_유저의_평균_주문액'] / orders['총주문액'] # 🐚 보통 평균이 뒤임. 개인 / 평균 

orders.sort_values('user_id',ascending=False)

,order_id,user_id,city,amount,해당_유저의_평균_주문액,총주문액,평균대비_비율
3,4,103,Seoul,5000,5000.0,5000,1.0
1,2,102,Busan,15000,20000.0,40000,0.5
4,5,102,Busan,25000,20000.0,40000,0.5
0,1,101,Seoul,10000,15000.0,30000,0.5
2,3,101,Seoul,20000,15000.0,30000,0.5
